In [15]:
# https://www.youtube.com/watch?v=W8ENIXvcGlQ&list=PLNEUSmZIK1CS16rElTshPGfMbgYrWQ4fm
# trend: 500 SMA over 200 SMA
# entry: 10 period RSI below 30, buy next day open
# exit: 10 period RSI above 40 OR sell after 10 trading days

In [16]:
import yfinance as yf
import pandas as pd
from tqdm import tqdm

data = yf.download("SPY", interval = "1d", period = "5y")
data.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2017-04-17,233.110001,234.570007,232.880005,234.570007,214.952881,68405400
2017-04-18,233.720001,234.490005,233.080002,233.869995,214.311417,83225800
2017-04-19,234.520004,234.949997,233.179993,233.440002,213.917374,68699900
2017-04-20,234.149994,235.850006,233.779999,235.339996,215.658493,92572200
2017-04-21,235.250000,235.309998,234.130005,234.589996,214.971191,110389800


In [17]:
from ta.momentum import rsi

#data.tail(1)

,Open,High,Low,Close,Adj Close,Volume,SMASMALL,RSI
Date,,,,,,,,
2022-04-13,438.029999,444.109985,437.839996,443.309998,443.309998,73965600,448.340351,46.965184


In [41]:
def oneSimulation(combinations):
    data, sma, rsival = combinations   
    data["SMASMALL"] = data["Close"].rolling(window = sma).mean()
    # data["SMABIG"] = data["Close"].rolling(window = 500).mean()
    data["RSI"] = rsi(data["Close"], rsival) 
    data = data.fillna(method="bfill")
    startMoney = 10000
    money = startMoney
    nrStocks = 0
    COMMISSION = 0.00025
    daysSincePurchase = 0

    trendActivated = False
    for i in range(len(data)):
        # first check if we have trend update
        crntPrice = data.iloc[i]["Close"]
        if crntPrice > data["SMASMALL"][i] and not trendActivated:
            trendActivated = True
        # that means now we are waiting for the 10 rsi to drop below 30
        if data["RSI"][i] < 30 and trendActivated and nrStocks == 0:
            # buy it
            howmany = money / crntPrice * .95
            money -= howmany * crntPrice * (1 + COMMISSION)
            nrStocks += howmany
            daysSincePurchase = 0
            trendActivated = False
        if nrStocks > 0:
            # we have to sell after 10 days or if rsi above 40
            if daysSincePurchase > 10 or data["RSI"][i] > 40:
                # sell it
                money += nrStocks * crntPrice * (1 - COMMISSION)
                nrStocks = 0
        daysSincePurchase += 1
    # on last day sell all stocks 4 info
    money += nrStocks * crntPrice * (1 - COMMISSION)
    nrStocks = 0

    #print("strategy earned %.2f$ in %d days" % (money - startMoney, len(data)))
    perMonth = (money - startMoney) / len(data) * 30 
    
    return perMonth, sma, rsival
ret, smaVal, rsiVal = oneSimulation((data, 200, 10))
print("%.2f$ per month" % ret)

14.84$ per month


In [42]:
from numpy.random import randint

SMAs = randint(0, 200, 50)
RSIs = randint(0, 30, 50)

combinations = [(data, sma, rsi) for sma in SMAs for rsi in RSIs]
print(len(combinations))

2500


In [43]:
from multiprocessing.pool import Pool

pool = Pool()
results = pool.map_async(oneSimulation, combinations).get()
print(results[0])

(37.84709976057847, 185, 23)


In [44]:
results = pd.DataFrame(results, columns=["perMonth", "sma", "rsi"])

In [47]:
results = results.sort_values(by="perMonth", ascending=False)
results.head(20)

,perMonth,sma,rsi
1493,58.813068,122,4
1479,58.813068,122,4
1477,58.813068,122,4
2193,58.813068,127,4
2180,58.813068,127,4
2179,58.813068,127,4
2177,58.813068,127,4
1480,58.813068,122,4
143,54.273365,129,4
1280,54.273365,129,4
